# LLM Chains

Chains allow us to combine multiple components together to create a single, coherent application. For example, we can create a chain that takes user input, formats it with a PromptTemplate, and then passes the formatted response to an LLM. We can build more complex chains by combining multiple chains together, or by combining chains with other components.

## Basic Single Chain

In [66]:
import os
from langchain_openai import ChatOpenAI
from langchain.prompts.chat import ChatPromptTemplate, HumanMessagePromptTemplate
from langchain.chains import LLMChain

In [67]:
human_message_prompt = HumanMessagePromptTemplate.from_template("Make up a funny company name for a company that produces {product}")
chat_prompt_template = ChatPromptTemplate.from_messages([human_message_prompt])

In [68]:
chat = ChatOpenAI(api_key=os.getenv("OPENAI_API_KEY"), model="gpt-4o")

In [69]:
chain = chat_prompt_template | chat
chain = LLMChain(llm=chat, prompt=chat_prompt_template)
response = chain.invoke(input="computers")

In [70]:
print(response['text'])

How about "Byte Me Computers"?


## Simple Sequential Chain

In [71]:
import os
from langchain_openai import ChatOpenAI
from langchain.prompts.chat import ChatPromptTemplate
from langchain.chains import LLMChain, SimpleSequentialChain

In [72]:
llm = ChatOpenAI(api_key=os.getenv("OPENAI_API_KEY"), model="gpt-4o")

In [73]:
template = "Give me a simple bullet point outline for a blog on post on {topic}"
first_prompt = ChatPromptTemplate.from_template(template=template)
chain_one = LLMChain(llm=llm, prompt=first_prompt)

template_2 = "Write a blog post using this outline: {outline}"
second_prompt = ChatPromptTemplate.from_template(template=template_2)
chain_two = LLMChain(llm=llm, prompt=second_prompt)

In [74]:
full_chain = SimpleSequentialChain(chains=[chain_one, chain_two], verbose=True)
result = full_chain.invoke(input="Data Science")
print(result)



> Entering new SimpleSequentialChain chain...
Sure! Here's a simple bullet point outline for a blog post on Data Science:

- **Introduction**
  - Brief explanation of what data science is
  - Importance and relevance of data science in today's world

- **Core Components of Data Science**
  - Data Collection
  - Data Cleaning
  - Exploratory Data Analysis (EDA)
  - Data Modeling
  - Data Visualization

- **Key Tools and Technologies**
  - Programming languages (Python, R)
  - Data manipulation tools (Pandas, NumPy)
  - Visualization libraries (Matplotlib, Seaborn)
  - Machine learning frameworks (Scikit-learn, TensorFlow)

- **Applications of Data Science**
  - Business Analytics
  - Healthcare
  - Finance
  - Marketing

- **Challenges in Data Science**
  - Data Quality and Volume
  - Privacy and Security Concerns
  - Keeping up with technological advancements

- **Future Trends in Data Science**
  - Rise of Automated Machine Learning (AutoML)
  - Increased use of Artificial Intellige

## Sequential Chains

In [88]:
import os
from langchain_openai import ChatOpenAI
from langchain.prompts.chat import ChatPromptTemplate
from langchain.chains import LLMChain, SequentialChain

In [89]:
llm = ChatOpenAI(api_key=os.getenv("OPENAI_API_KEY"), model="gpt-4o")

In [90]:
template_1 = "Give a summary of this employee's performance review:\n{review}"
prompt_1 = ChatPromptTemplate.from_template(template=template_1)
chain_1 = LLMChain(llm=llm, prompt=prompt_1, output_key="review_summary")

In [91]:
template_2 = "Identify key employee weaknesses in this review summary:\n{review_summary}"
prompt_2 = ChatPromptTemplate.from_template(template=template_2)
chain_2 = LLMChain(llm=llm, prompt=prompt_2, output_key="weaknesses")

In [92]:
template_3 = "Create a personalized plan to help address and fix these weaknesses:\n{weaknesses}"
prompt_3 = ChatPromptTemplate.from_template(template=template_3)
chain_3 = LLMChain(llm=llm, prompt=prompt_3, output_key="final_plan")

In [93]:
seq_chain = SequentialChain(chains=[chain_1, chain_2, chain_3], 
                            input_variables=['review'],
                            output_variables=['review_summary','weaknesses','final_plan'],
                            verbose = True)

In [94]:
employee_review = '''
Employee Information:
Name: Joe Schmo
Position: Software Engineer
Date of Review: July 14, 2023

Strengths:
Joe is a highly skilled software engineer with a deep understanding of programming languages, algorithms, and software development best practices. His technical expertise shines through in his ability to efficiently solve complex problems and deliver high-quality code.

One of Joe's greatest strengths is his collaborative nature. He actively engages with cross-functional teams, contributing valuable insights and seeking input from others. His open-mindedness and willingness to learn from colleagues make him a true team player.

Joe consistently demonstrates initiative and self-motivation. He takes the lead in seeking out new projects and challenges, and his proactive attitude has led to significant improvements in existing processes and systems. His dedication to self-improvement and growth is commendable.

Another notable strength is Joe's adaptability. He has shown great flexibility in handling changing project requirements and learning new technologies. This adaptability allows him to seamlessly transition between different projects and tasks, making him a valuable asset to the team.

Joe's problem-solving skills are exceptional. He approaches issues with a logical mindset and consistently finds effective solutions, often thinking outside the box. His ability to break down complex problems into manageable parts is key to his success in resolving issues efficiently.

Weaknesses:
While Joe possesses numerous strengths, there are a few areas where he could benefit from improvement. One such area is time management. Occasionally, Joe struggles with effectively managing his time, resulting in missed deadlines or the need for additional support to complete tasks on time. Developing better prioritization and time management techniques would greatly enhance his efficiency.

Another area for improvement is Joe's written communication skills. While he communicates well verbally, there have been instances where his written documentation lacked clarity, leading to confusion among team members. Focusing on enhancing his written communication abilities will help him effectively convey ideas and instructions.

Additionally, Joe tends to take on too many responsibilities and hesitates to delegate tasks to others. This can result in an excessive workload and potential burnout. Encouraging him to delegate tasks appropriately will not only alleviate his own workload but also foster a more balanced and productive team environment.
'''

In [99]:
results = seq_chain.invoke(employee_review)
results



> Entering new SequentialChain chain...

> Finished chain.


{'review': "\nEmployee Information:\nName: Joe Schmo\nPosition: Software Engineer\nDate of Review: July 14, 2023\n\nStrengths:\nJoe is a highly skilled software engineer with a deep understanding of programming languages, algorithms, and software development best practices. His technical expertise shines through in his ability to efficiently solve complex problems and deliver high-quality code.\n\nOne of Joe's greatest strengths is his collaborative nature. He actively engages with cross-functional teams, contributing valuable insights and seeking input from others. His open-mindedness and willingness to learn from colleagues make him a true team player.\n\nJoe consistently demonstrates initiative and self-motivation. He takes the lead in seeking out new projects and challenges, and his proactive attitude has led to significant improvements in existing processes and systems. His dedication to self-improvement and growth is commendable.\n\nAnother notable strength is Joe's adaptability.

In [96]:
results

{'review': "\nEmployee Information:\nName: Joe Schmo\nPosition: Software Engineer\nDate of Review: July 14, 2023\n\nStrengths:\nJoe is a highly skilled software engineer with a deep understanding of programming languages, algorithms, and software development best practices. His technical expertise shines through in his ability to efficiently solve complex problems and deliver high-quality code.\n\nOne of Joe's greatest strengths is his collaborative nature. He actively engages with cross-functional teams, contributing valuable insights and seeking input from others. His open-mindedness and willingness to learn from colleagues make him a true team player.\n\nJoe consistently demonstrates initiative and self-motivation. He takes the lead in seeking out new projects and challenges, and his proactive attitude has led to significant improvements in existing processes and systems. His dedication to self-improvement and growth is commendable.\n\nAnother notable strength is Joe's adaptability.

In [98]:
print(results['review'])


Employee Information:
Name: Joe Schmo
Position: Software Engineer
Date of Review: July 14, 2023

Strengths:
Joe is a highly skilled software engineer with a deep understanding of programming languages, algorithms, and software development best practices. His technical expertise shines through in his ability to efficiently solve complex problems and deliver high-quality code.

One of Joe's greatest strengths is his collaborative nature. He actively engages with cross-functional teams, contributing valuable insights and seeking input from others. His open-mindedness and willingness to learn from colleagues make him a true team player.

Joe consistently demonstrates initiative and self-motivation. He takes the lead in seeking out new projects and challenges, and his proactive attitude has led to significant improvements in existing processes and systems. His dedication to self-improvement and growth is commendable.

Another notable strength is Joe's adaptability. He has shown great flexi

In [100]:
print(results['review_summary'])

Joe Schmo, a Software Engineer, received a performance review on July 14, 2023, highlighting both his strengths and areas for improvement. Joe is recognized for his technical expertise in software development, his collaborative nature, and his initiative and adaptability. He excels in problem-solving and effectively engages with cross-functional teams, often leading to improvements in processes and systems. 

However, there are a few areas where Joe could enhance his performance. He occasionally struggles with time management, leading to missed deadlines, and his written communication skills sometimes lack clarity. Additionally, Joe tends to take on too many responsibilities without delegating, which can lead to an excessive workload and potential burnout. Improving his time management, written communication, and delegation skills would enhance his efficiency and contribute to a more balanced team environment.


In [101]:
print(results['weaknesses'])

In the performance review summary, Joe Schmo's key weaknesses are identified as follows:

1. **Time Management**: Joe occasionally struggles with managing his time effectively, which has resulted in missed deadlines.

2. **Written Communication**: His written communication skills sometimes lack clarity, which may impact the effectiveness of his interactions and documentation.

3. **Delegation**: Joe tends to take on too many responsibilities without delegating tasks, leading to an excessive workload and potential burnout. Improving his ability to delegate could help distribute the workload more evenly and prevent overburdening himself.


In [102]:
print(results['final_plan'])

To address and improve Joe Schmo's identified weaknesses, a personalized development plan can be created. This plan will include specific actions, resources, and timelines to guide Joe's growth in these areas. Here is a detailed plan for each of the three key areas:

### 1. Time Management
**Objective**: Improve Joe's ability to manage his time effectively to meet deadlines consistently.

**Actions**:
- **Prioritization Training**: Enroll Joe in a time management workshop or online course focusing on prioritization techniques (e.g., Eisenhower Box, ABC prioritization).
- **Daily Planning**: Encourage Joe to spend 10-15 minutes at the start of each day planning and prioritizing his tasks. He can use tools like digital calendars or planning apps (e.g., Trello, Todoist).
- **Pomodoro Technique**: Introduce the Pomodoro Technique to help Joe maintain focus and efficiency during work hours. This involves working in focused sprints of 25 minutes followed by a 5-minute break.
- **Weekly Revie